# ECON 409: Final Project
#### Courtney Manhart, Tanner Woods, Lindsay Mahowald, and Martin Jamouss

In [1]:
# Import Necessary Packages
import yfinance as yf 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import io
import matplotlib.lines as mlines
import statsmodels.formula.api as smf 
from statsmodels.tsa.arima.model import ARIMA
import scipy.stats as st
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess 
import pmdarima as pm 
from statsmodels.graphics.tsaplots import plot_predict

### Import the Data: Choose one or more series of economic fundamentals to use for generating exchange rate forecasts. The forecasts of the asset price may be directional or point forecasts

###### Economic Fundamentals: Inflation, Interest rates, GDP

### Outline Trading Rules based on these forecasts:
- Long/short positions
- exiting positions

In [ ]:
### Long / Short Postions:
    # Long:
    # Short: 


    
    
    
# Exiting Positions:


### Optimize at least 2 strategy Hyperparameters:
- length of holding period
- model hyperparameters (alpha for lasso, number of lags to include, rolling window size, etc)

### Visualize Optimization process using a heatmap

In [ ]:
# Heatmaps

def heatmap(x, y, metric, values):
    
    # specify the columns I will be pulling from the results
    p2p = values[[x, y, metric]]
    
    # If p > 2, we need to group 
    heat = np.round(p2p.groupby([x,y]).max(),1)
    heat = heat.unstack()[metric]

    # round labels
    heat.index = np.round(heat.index,2)
    heat.columns = np.round(heat.columns,2)

    # make plot
    f, ax = plt.subplots(figsize=(15, 8))
    ax = sns.heatmap(heat, fmt='.1g')
    ax.set_title("Heat map of "+ metric,size = 15)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.set_xlabel(y, size = 15)
    ax.set_ylabel(x, size = 15)
    ax.collections[0].colorbar.set_label(metric, size = 15)
    sns.set(font_scale=1)
    plt.show()

In [ ]:
# Heatmap for Hyperparameters
heatmap()

### Trading Strategy: using the optimal results

### Produce Equity Curve over a backtest

### Use the Binomial Test (directional forecast) or the DMW Test (point forecast) 

### Produce table with HFRI Metrics to evaluare the strategy's performance

#### Setting up the Tables:

In [ ]:
## Risk/Return Table
index = ['Geo. Average Monthly', 'Std. Deviation', 'High Month',
        'Low Month', 'Annualized Return', 'Annualized STD', 
        'Risk Free Rate', 'Sharpe Ratio', '% of Winning Mo.',
        'Max Drawdown']
columns = ['Strategy']
risk_return_table = pd.DataFrame(index = index, columns = columns)

In [ ]:
## Regresssion Table:
index = ['Alpha', 'Beta', 'Mnt. R-Squared', 'Correlation', 
         'Up Alpha', 'Up Beta', 'Up R-Squared', 'Down Alpha',
        'Down Beta', 'Down R-Squared']
columns = ['Strategy']
reg_table = pd.DataFrame(index = index, columns = columns)

#### Regression Results

*Alpha & Beta*

In [ ]:
reg_table.at['Alpha', 'Strategy'] = alpha
reg_table.at['Beta', 'Strategy'] = beta

*Mnt. R-Squared*

In [ ]:
reg_table.at['Mnt. R-Squared', 'Strategy'] = r_sq

*Correlation*

In [ ]:
reg_table.at['Correlation', 'Strategy'] = corr

*Up Alpha, Up Beta & Up R-Squared*

In [ ]:
reg_table.at['Up Alpha', 'Strategy'] = up_alpha
reg_table.at['Up Beta', 'Strategy'] = up_beta
reg_table.at['Up R-Squared', 'Strategy'] = up_r_sq

*Down Alpha, Down Beta & Down R-Squared*

In [ ]:
reg_table.at['Down Alpha', 'Strategy'] = down_alpha
reg_table.at['Down Beta', 'Strategy'] = down_beta
reg_table.at['Down R-Squared', 'Strategy'] = dow_r_sq

#### Risk/Return Results

*Geometric Average Monthly & Standard Deviation*

In [ ]:
risk_retun_table.at['Geo. Average Monthly', 'Strategy'] = geo_avg
risk_retun_table.at['Std. Deviation', 'Strategy'] = stdev

*High Month & Low Month*

In [ ]:
risk_retun_table.at['High Month', 'Strategy'] = high_month
risk_retun_table.at['Low Month', 'Strategy'] = low_month

*Annualized Return & Standard Deviation*

In [ ]:
risk_retun_table.at['Annualized Return', 'Strategy'] = ann_ret
risk_retun_table.at['Annualzed STD', 'Strategy'] = ann_std

*Risk Free Rate*

In [ ]:
risk_retun_table.at['Risk Free Rate', 'Strategy'] = rfr

*Sharpre Ratio*

In [ ]:
risk_retun_table.at['Sharpe Ratio', 'Strategy'] = sharpe_ratio

*% of Winning Mo.*

In [ ]:
risk_retun_table.at['% of Winning Mo.', 'Strategy'] = win_mo

*Max Drawdown*

In [ ]:
risk_retun_table.at['Max Drawdown', 'Strategy'] = max_draw

#### Final Tables

In [ ]:
risk_return_table

In [ ]:
reg_table